In [2]:
import pandas as pd

In [ ]:
dataset="datasets/"
folder="energy/" 

In [17]:
df=pd.read_csv(folder + "cleaned_dataset.csv")

In [ ]:
df = df[~df["post_type"].isna()]
df=df[~df["created_at"].isna()]

In [ ]:
df=df[df["post_type"]=="retweet"]

In [53]:
import pandas as pd
import re

words1 = ["Baker Hughes","BKR.O","Venture Global","VG.N","Plaquemines","Calcasieu Pass","EXXON","$XOM","Repsol","$REPYY","Hanwha","$KRW","Argent LNG","Woodside Energy","$WDS","Cheniere","ConocoPhillips","$COP","Petronas","PNAGF"]
words2 = ["gas","plant","drill"]

def fast_match(text):
    if pd.isna(text):
        return False
    text1 = text.lower()
    for word1 in words1:
        if word1.lower() in text1:
            if re.search(r'\b' + re.escape(word1.lower()) + r'\b', text1):
                return True
    return False

camp1 = df[df['content'].apply(fast_match)].copy()

In [55]:
import pandas as pd
import re

words1 = ["BP.L","SHEL.L","Edison SpA","EDNn.MI","TotalEnergies","$TTE","Equinor","EQNR","Chevron","CVX","ADNOC","SOCAR","KRX","QatarEnergy","QFLS","CNOOC","PetroChina","Gasprom","Novatek"]
words2 = ["gas","plant","drill","NewMed"]

def fast_match(text):
    if pd.isna(text):
        return False
    text1 = text.lower()
    for word1 in words1:
        if word1.lower() in text1:
            if re.search(r'\b' + re.escape(word1.lower()) + r'\b', text1):
                return True
    return False

camp2_1 = df[df['content'].apply(fast_match)].copy()

In [57]:
import pandas as pd
import re

words1 = ["BP","Shell"]
words2 =["gas","plant","drill","NewMed"]

def fast_match(text):
    if pd.isna(text):
        return False
    text1 = text.lower()
    for word1 in words1:
        if word1.lower() in text1:
            if re.search(r'\b' + re.escape(word1.lower()) + r'\b', text1):
                for word2 in words2:
                    if re.search(r'\b' + re.escape(word2.lower()) + r'\b', text1):
                        return True
    return False

camp2_2 = df[df['content'].apply(fast_match)].copy()

In [ ]:
camp2=pd.concat([camp2_1, camp2_2], ignore_index=True) 

In [60]:
camp2=camp2.drop_duplicates(subset='item_id', keep='first')

In [62]:
import pandas as pd
import re

words1 = ["UAE","QATAR","Saudi Arabia","Russia","China","EU","Europe","Egypt","Israel","America"]
words2 = ["gas"]

def fast_match(text):
    if pd.isna(text):
        return False
    text1 = text.lower()
    for word1 in words1:
        if word1.lower() in text1:
            if re.search(r'\b' + re.escape(word1.lower()) + r'\b', text1):
                for word2 in words2:
                    if re.search(r'\b' + re.escape(word2.lower()) + r'\b', text1):
                        return True
    return False

camp3 = df[df['content'].apply(fast_match)].copy()

In [ ]:
lng_salient=pd.concat([camp1, camp2,camp3], ignore_index=True) 
lng_salient.shape

In [65]:
lng_salient=lng_salient.drop_duplicates(subset='item_id', keep='first')

In [ ]:
energy_users=lng_salient["user_id"].to_list()+lng_salient["parent_user_id"].to_list()

In [ ]:
energy_tweets=lng_salient["item_id"].to_list()+lng_salient["parent_item_id"].to_list()

In [ ]:
energy_df = pd.DataFrame({'Id': list(set(energy_tweets))})
energy_df.to_csv(folder+"graph/lng_salient_tweet_ids.csv", index=False)

In [ ]:
lng_salient.to_csv(folder+"graph/lng_salient_retweets.csv", index=False)

In [21]:
lng_salient=pd.read_csv(folder+"graph/lng_salient_retweets.csv")

In [ ]:
energy_df = pd.DataFrame({'Id': list(set(energy_users)), 'type': 'energy'})
energy_df.to_csv(folder+"graph/energy_users.csv", index=False)

In [73]:
df=df=pd.read_csv(folder + "cleaned_dataset.csv")

In [ ]:
import networkx as nx
print(df.shape)
df = df[df['post_type'] == 'retweet']
print(df.shape)
df = df.dropna(subset=['user_id', 'parent_user_id'])
print(df.shape)

df = df[(df['user_id'] != '') & (df['parent_user_id'] != '')]
print(df.shape)
df = df[(df['user_id'].notna()) & (df['parent_user_id'].notna())]
print(df.shape)

a=set()
graph = nx.Graph()  
for _, row in df.iterrows():
    user_id = row['user_id']
    a.add(user_id)
    parent_user_id = row['parent_user_id']
    a.add(parent_user_id)
    if graph.has_edge(user_id, parent_user_id):
        graph[user_id][parent_user_id]['weight'] += 1
    elif graph.has_edge(parent_user_id, user_id):
        graph[parent_user_id][user_id]['weight'] += 1
    else:
        graph.add_edge(user_id, parent_user_id, weight=1)
#edges_to_remove = [(u, v) for u, v, d in graph.edges(data=True) if d['weight'] < edge_threshold]   
#graph.remove_edges_from(edges_to_remove)
edge_list = [(u, v, d['weight']) for u, v, d in graph.edges(data=True)]
edges_df = pd.DataFrame(edge_list, columns=['source', 'target', 'weight'])
edges_df.to_csv(folder+"graph/graph_all.csv", index=False)
nx.write_gexf(graph,folder+"graph/all.gexf")
print(len(a))
print(f"Total unique users: {len(a)}")
print(f"Graph edges saved to graph_all.csv with {len(edge_list)} edges.")

In [ ]:
import pandas as pd
energy_users=folder+"graph/energy_users.csv"
energy_users=pd.read_csv(energy_users)
energy_users=energy_users["Id"].to_list()


In [ ]:
users_df=pd.read_csv(folder+"graph/energy_users.csv")
user_list=users_df["Id"].to_list()

In [ ]:
import networkx as nx
import pandas as pd
from community import community_louvain

def louvain_algorithm(graph_G: nx.Graph, resolution: float) -> pd.DataFrame:
    partition = community_louvain.best_partition(graph_G, weight='weight', resolution=resolution, random_state=42)
    return pd.DataFrame(partition.items(), columns=['node_id', 'community'])

def compute_common_users(user_community_df: pd.DataFrame, user_list: list):
    community_stats = user_community_df.groupby('community')['node_id'].apply(list).reset_index()
    community_stats['common_user_count'] = community_stats['node_id'].apply(lambda x: len(set(x) & set(user_list)))
    mean_plus_sigma = community_stats['common_user_count'].mean() + community_stats['common_user_count'].std()
    print(mean_plus_sigma)
    filtered_communities = community_stats[community_stats['common_user_count'] > mean_plus_sigma]['community']
    return user_community_df[user_community_df['community'].isin(filtered_communities)]

def export_filtered_graph(G: nx.Graph, user_community_df: pd.DataFrame, nodes_csv: str, edges_csv: str):
    subgraph_nodes = set(user_community_df['node_id'])
    subgraph = G.subgraph(subgraph_nodes).copy()
    nodes_df = pd.DataFrame(list(subgraph.nodes(data=True)), columns=['node_id', 'attributes'])
    nodes_df['community'] = nodes_df['node_id'].map(user_community_df.set_index('node_id')['community'])
    nodes_df = pd.concat([nodes_df.drop(['attributes'], axis=1), nodes_df['attributes'].apply(pd.Series)], axis=1)
    edges_df = pd.DataFrame(list(subgraph.edges(data=True)), columns=['source', 'target', 'attributes'])
    edges_df = pd.concat([edges_df.drop(['attributes'], axis=1), edges_df['attributes'].apply(pd.Series)], axis=1)
    nodes_df.to_csv(nodes_csv, index=False)
    edges_df.to_csv(edges_csv, index=False)

def driver(input_file: str, user_list: list, nodes_csv: str, edges_csv: str):
    G = nx.read_gexf(input_file)
    user_community_df = louvain_algorithm(G, resolution=1.0)
    filtered_user_community_df = compute_common_users(user_community_df, user_list)
    export_filtered_graph(G, filtered_user_community_df, nodes_csv, edges_csv)

if __name__ == "__main__":
    input_file=folder+"graph/all.gexf"
    nodes_csv=folder+"graph/lv1_filtered_nodes.csv"
    edges_csv=folder+"graph/lv1_filtered_edges.csv"
    driver(input_file, user_list, nodes_csv, edges_csv)